In [455]:
import sklearn
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import re
import math
import warnings
import os
import math
import matplotlib.pyplot as plt
import socket
import tempfile
warnings.filterwarnings('ignore')

from tqdm import tqdm
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
import sklearn.cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, classification_report, roc_auc_score 
from alibi.explainers import AnchorTabular
from sklearn.neighbors import NearestNeighbors, KDTree, NearestCentroid
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier



class ModelGlobalExplainer():
    def __init__(self,
                 ):
        pass
    
    def fit(self,
            x,
            y,
           point_no_to_global):
        
        self.explainer = DecisionTreeClassifier(random_state=0,
                                                max_depth = math.floor(math.log2(point_no_to_global)))
        self.explainer.fit(x,y)
        pass
    
    def predict(self,
                X_test, 
                features # keep to be consistent with modellocalexplainer
               ):
        
        self.y_pred = self.explainer.predict(X_test)
        pass

class ModelLocalExplainer():
    def __init__(self, 
        clf: sklearn.base.BaseEstimator,
        class_names: list):
        self.clf = clf
        self.class_names = class_names
        pass
        
    def fit(self, 
            dataset:pd.DataFrame, #dataset that will be used to train the explainer 
            features: list,
            thresh: float
            #target:pd.Series # column with target values
            ):
        
        explainer = self._fit_explainer(dataset.iloc[:,:-1], features)
        self.rules = self._justify_explainer(dataset, explainer, thresh)
        self.hmr_file_name = self.df2hmr(data = self.rules.drop('mult', axis = 1), df_columns_names = features)
        pass

    #def justify(self, 
    #    instance_to_explain:pd.DataFrame, # vector of values representing a single instane to explain (e.g. centroid)
    #    **kwargs:dict # additional arguments for the explainer to generate explanations
    #    ):
    #    """ Returns textual explanation (e.g. rule in text format)
    #    """
    #    #self.result = self._convert_rules(self.rules, self.orignal_features, self.features)
    #    pass
    
    

    def predict(self, X_test, features):
        self.y_pred = self._heartdroid(X_test, features)
        pass

    ###HMR file creation
    def inparse(self, condition):
        fs = re.sub(r'([-+]?[0-9]+\.[0-9]+)(<=|>=|<|>|)(f[0-9]+)(<=|>=|<|>)([-+]?[0-9]+\.[0-9]+)',r'\2',condition)
        ss =re.sub(r'([-+]?[0-9]+\.[0-9]+)(<=|>=|<|>|)(f[0-9]+)(<=|>=|<|>)([-+]?[0-9]+\.[0-9]+)',r'\4',condition)
        res=None
        if fs == '<':
            val = re.sub(r'([-+]?[0-9]+\.[0-9]+)(<=|>=|<|>|)(f[0-9]+)(<=|>=|<|>)([-+]?[0-9]+\.[0-9]+)',r'\1',condition)
            res = re.sub(r'([-+]?[0-9]+\.[0-9]+)(<=|>=|<|>|)(f[0-9]+)(<=|>=|<|>)([-+]?[0-9]+\.[0-9]+)',r'\3 in ['+str(eval(val)+0.001)+r' to \5 ]',condition)
        if ss == '<':
            val = re.sub(r'([-+]?[0-9]+\.[0-9]+)(<=|>=|<|>|)(f[0-9]+)(<=|>=|<|>)([-+]?[0-9]+\.[0-9]+)',r'\5',condition)
            res = re.sub(r'([-+]?[0-9]+\.[0-9]+)(<=|>=|<|>|)(f[0-9]+)(<=|>=|<|>)([-+]?[0-9]+\.[0-9]+)',r'\3 in [\1 to '+str(eval(val)-0.001)+']',condition)
        if res is None:
            return condition
        else:
            return res

    def tohmr(self, series):
        result =[]
        for v in  series.split('AND'):
            v = self.inparse(v.strip().lower().replace(' ',''))
            result.append(v.replace('<=',' lte ')
                          .replace('>=',' gte ').replace('<',' lt ').replace('>',' gt ').replace('=','eq').lower())
        return '['+','.join(result)+']'

    def df2hmr(self, data, df_columns_names):
        numfeats = len(df_columns_names)
        
        types = """xtype [name: float,
    domain: [-10000 to 10000],
    scale: 0,
    base: numeric
    ].
xtype [name: clustertype,
    domain: [0 to 1000],
    scale: 0,
    base: numeric
    ].
"""
        atts_cluster = """
xattr [name: cluster,
    type: clustertype,
    class: simple,
    comm: out
    ].
"""
        atts_placeholder = """
xattr [name: __NAME__,
    type: float,
    class: simple,
    comm: out
    ].
"""
        schema_placeholder = """xschm anchor: [__NAME__] ==> [cluster].
"""
        data['hmr_cond'] = data['Rule'].apply(self.tohmr)  
        data['confidence'] = data['Coverage']*data['Precision']
        atts = ''
        schemacond = []
        #for i in range(1,numfeats+1):
        #    atts+=atts_placeholder.replace('__NAME__','f'+str(i))
        #    schemacond.append('f'+str(i))
        for i in df_columns_names:
            atts+=atts_placeholder.replace('__NAME__',i)
            schemacond.append(i)        
        
        #print(schemacond)
        schema = schema_placeholder.replace('__NAME__',','.join(schemacond))
        
        #print(schema)
        """
        with open('model.hmr','w') as f:
            f.write(types)
            f.write(atts) 
            f.write(atts_cluster) 
            f.write(schema) 
            for i,r in data.iterrows():
                f.write('xrule anchor/'+str(i)+': '+r['hmr_cond']+ ' ==>  [cluster set '+str(r['Cluster'])+']. #'+str(r['confidence'])+'\n')
        """
        f = tempfile.NamedTemporaryFile(mode = 'w', delete=False, suffix='.hmr', dir=os.getcwd())
        #config_path = f'{f.name}.hmr'
        #with (config_path,'w') as f:
        f.write(types)
        f.write(atts) 
        f.write(atts_cluster) 
        f.write(schema) 
        for i,r in data.iterrows():
            f.write('xrule anchor/'+str(i)+': '+r['hmr_cond']+ ' ==>  [cluster set '+str(r['Cluster'])+']. #'+str(r['confidence'])+'\n')
        f.close()
        #print(f.name)
        return f.name.split('/')[-1]
        
        
                
    def _heartdroid(self, X_test_con, features):
        temp_list = []  
        X_test = X_test_con.values
        model = self.hmr_file_name
        
        print('Heartdroid run')
        
        for steps in tqdm(range(X_test.shape[0]), desc="Loading...", position=0, leave=True):
            finall_string = ''
            for it, index in enumerate(features):
                finall_string += f' -A {index}={X_test[steps][it]}'
                
            #output_list = !java -jar HMRCommandLine.jar {model} -tabs anchor{finall_string}
            output_list = queryHRTDServer(f'{model} -tabs anchor{finall_string}')
            output_list = output_list.split('\n')
            
            for o in reversed(output_list):
                if 'Attribute cluster' in o:
                    output = [o]
                
            if 'null' in output[0]:
                temp_list.append(-1) #undefined cluster
            else:
                temp_list.append(int(float(output[0].split(" = ")[-1])))
        temp_list = np.array(temp_list)
        return temp_list

class ModelLocalExplainer_anchor(ModelLocalExplainer):    
    
    def _fit_explainer(self, dataset, features):
        predict_fn = lambda x: self.clf.predict_proba(x)
        explainer = AnchorTabular(predict_fn, features)
        explainer.fit(dataset.values, disc_perc=(25, 50, 75))
        return explainer

    def _justify_explainer(self, dataset, explainer, thresh):
        rules_out_list = []
        for cluster in self.class_names:
            rules_out = pd.DataFrame()
            tempo_dataset = dataset[dataset['y'] == cluster]
            for idx in range(tempo_dataset.shape[0]):
                if self.class_names[explainer.predictor(tempo_dataset.iloc[:,:-1].values[idx].reshape(1, -1))[0]] == cluster:
                    explanation = explainer.explain(tempo_dataset.iloc[:,:-1].values[idx], threshold=thresh)
                    exp = explanation.anchor
                    rules_out = rules_out.append({'Rule': (' AND '.join(exp)), 
                                                  'Precision': explanation['precision'], 
                                                  'Coverage': explanation['coverage'], 
                                                  'Cluster': cluster}, 
                                                  ignore_index = True)
            rules_out['mult'] = rules_out['Precision'] * rules_out['Coverage']
            rules_out_list.append(rules_out.sort_values('mult', ascending = False).drop_duplicates(subset=['Rule']).reset_index(drop = True))
               
        rules_output = pd.concat(rules_out_list)
        rules_output.reset_index(drop = True, inplace = True)
        return rules_output

class CLAMP(BaseEstimator):
    def __init__(self,
        bounding_box_selection: str='random',
        classification_model: sklearn.base.BaseEstimator = LogisticRegression(), 
        clusterng_algorithm: sklearn.base.BaseEstimator = KMeans(),
        description_points_ratio: float=0.1,
        test_size: float=0.2,
        metric: str='minkowski',
        explainer_type: str='anchor',
        thresh: float=0.9,
        conv_method: str = None
        ):
        
        self.bounding_box_selection = bounding_box_selection
        self.classification_model = classification_model
        self.clusterng_algorithm = clusterng_algorithm
        self.description_points_ratio = description_points_ratio
        self.test_size = test_size
        self.metric = metric
        self.explainer_type=explainer_type
        self.thresh = thresh
        self.hrd_accuracy = 0
        self.conv_method = None
        
        pass
         
    def fit(self,
        x_in : pd.DataFrame, #data which will be used to train explainer model and classifier
        y: pd.Series=None, # cluster labels (not used, left for consistency with BaseEstimator)
        ):
        """
        #fits the Clustering algorithm and classifier
        """
        
        #exchange column names
        x, self.orignal_features = self._convert_features(x_in)
        if y is None:
            #clustering stage
            y = self._clustering(x) # only if y not in data
            print('Data without labels, clustering stage implementation')
        else:
            y = np.array(y)
            print('Data labeled')
            
        #classification stage
        self.X_train, self.X_test, self.y_train, self.y_test = self._recognize_input(x, y)
        self.X_train = self._convert_to_norm(self.X_train, self.conv_method)
        self.X_test = self._convert_to_norm(self.X_test, self.conv_method)
        self.y_pred_clf, self.clf_model = self._classification()
        self.clf_precision, self.clf_recall, self.clf_f1, self.clf_accuracy, self.clf_classification_report = self._scores(self.y_test, self.y_pred_clf)        
        
        #bounding box stage
        self.df_model_input =  self._bounding_box_method(self.X_train, self.y_train)
        #print(math.floor(math.log2(self.point_no_to_global)+1))
        
        if self.explainer_type == 'anchor':
            self.explainer = ModelLocalExplainer_anchor(clf = self.clf_model, class_names = self.class_names)
            print('Anchor explainer')
            self.explainer.fit(dataset=self.df_model_input, features = self.features, thresh = self.thresh)
            
        elif self.explainer_type == 'global':
            self.explainer = ModelGlobalExplainer()
            print('DT explainer')
            self.explainer.fit(x=self.X_train, y=self.y_train, point_no_to_global = self.point_no_to_global)
        else:
            ValueError('Explainer type not implemented. Select one of: anchor, global.')
        pass
        
        
    def justify(self):#, X_test, y_test):
        #self.explainer.justify(X_test, y_test)
        #self.results = self._convert_rules(self.explainer.rules.drop('mult', axis = 1), self.orignal_features, self.features)
        return self._convert_rules(self.explainer.rules.drop('mult', axis = 1), self.orignal_features, self.features)
    
    def predict(self, X_test, y_test):
        self.explainer.predict(X_test, self.features)
        self.y_pred_explainer = self.explainer.y_pred
        self.explainer_precision, self.explainer_recall, self.explainer_f1, self.explainer_accuracy, self.explainer_classification_report = self._scores(y_test, self.y_pred_explainer)
        pass
    
    def _convert_rules (self, rules, org, mod):
        my_dict = {} 
        for key in mod: 
            for value in org: 
                my_dict[key] = value 
                org.remove(value) 
                break 
        rules_con_org_features = []
        for rule in rules['Rule']:
            temp = rule.split()
            res = []
            for wrd in temp:
                res.append(my_dict.get(wrd, wrd))
            rules_con_org_features.append(' '.join(res))
        rules['Rule'] = rules_con_org_features
        return rules
    
    def _convert_features(self, data):
        orignal_features = list(data.columns)
        data.columns = ['f' + str(list(data.columns).index(x)) for x in list(data.columns)]
        return data, orignal_features
    
    def _clustering(self, x):       
        try:
            clustering_model = self.clusterng_algorithm.fit(x)
            prediction = clustering_model.predict(x)
        except:
            prediction = self.clusterng_algorithm.fit_predict(x)
        return prediction
    
    def _recognize_input(self, x, y):
        self.features = x.columns
        self.num_of_features = len(self.features)
        self.class_names = np.unique(y)
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=self.test_size, random_state=0)
        return X_train, X_test, y_train, y_test
    
    
    ####not used in grid search
    def _convert_to_norm(self,x, method):
        if method == 'standard_scaler':
            scaler = StandardScaler()
            scaler.fit(x)
            converted_data = scaler.transform(x)
        elif method == 'minmax_scaler':
            scaler = MinMaxScaler()
            scaler.fit(x)
            converted_data = scaler.transform(x)
        elif method == None:
            converted_data = x
        return converted_data
    
    def _classification(self):   
        classification_model = self.classification_model.fit(self.X_train, self.y_train)
        return classification_model.predict(self.X_test), classification_model
    
    def _scores(self, y_test, y_pred):
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        accuracy = accuracy_score(y_test, y_pred)
        classification_rep = classification_report(y_test, y_pred, labels=self.class_names)
        return precision, recall, f1, accuracy, classification_rep#, auc
    
    def _bounding_box_method(self, x, y):
        data = pd.concat([x.reset_index(drop = True), pd.Series(y)], axis=1)
        data.columns = list(self.features)+['y']
        number_of_points = math.ceil(data.shape[0]*self.description_points_ratio)
        self.point_no_to_global = number_of_points
        #print(data.shape)
        temp_list = []
        if self.bounding_box_selection == 'random':
            for cluster in self.class_names:
                X_t = data[data['y'] == cluster]
                try:
                    temp_list.append(X_t.sample(n = number_of_points))
                except:
                    temp_list.append(X_t.sample(n = X_t.shape[0]))
                    
        elif self.bounding_box_selection == 'tree_query':
            clf = NearestCentroid()
            clf.fit(data.drop('y', axis = 1).to_numpy(), data['y'].to_numpy())
            df_centroids = pd.DataFrame(clf.centroids_, columns = self.features)
            for cluster in self.class_names:
                X_t = data[data['y'] == cluster].drop('y', axis = 1).values
                tree = KDTree(X_t, leaf_size = 10, metric = self.metric)  
                dist, ind = tree.query(df_centroids.iloc[cluster].values.reshape(1,-1), k=len(X_t))      
                temp_df = data[data['y'] == cluster].iloc[ind[0][-number_of_points:],:]
                temp_list.append(temp_df)
        
        elif self.bounding_box_selection == 'outliers':
            for cluster in self.class_names:
                cluster_cont = number_of_points/(data[data['y'] == cluster].shape[0])
                if cluster_cont > 0.5:
                    print("number of description points cannot be higher than 50%, value has been changed to maximum")
                    cluster_cont = 0.5
                random_data = data[data['y'] == cluster].drop('y', axis = 1).values
                clf = IsolationForest(random_state = 0, contamination= cluster_cont)
                preds = clf.fit_predict(random_data)
                df_temp = pd.DataFrame(random_data[[i for i, x in enumerate(preds) if x == -1]], columns = self.features)
                df_temp['y'] = [cluster] * len(df_temp)
                temp_list.append(df_temp)
                
        elif self.bounding_box_selection == 'centroids':
            for cluster in self.class_names:
                X_t = data[data['y'] == cluster].drop('y', axis = 1)
                X_t = X_t.to_numpy()
                kmedoids = KMedoids(n_clusters=1, random_state=0).fit(X_t)
                
                df_temp = pd.DataFrame(data = kmedoids.cluster_centers_, columns = data.drop('y', axis = 1).columns)
                df_temp['y'] = [cluster] * len(df_temp)
                temp_list.append(df_temp)
        else:
            ValueError('Bounding box method not implemented. Select one of: random, tree_query, outliers.')
        df_ready = pd.concat(temp_list)
        df_ready.reset_index(inplace = True, drop = True)
        return df_ready
 
    def _create_necessary_dir(self):
        path = pathlib.Path().resolve() / 'hmr_models'
        isExist = os.path.exists(path)
        if not isExist:
            
          os.makedirs(path)

In [564]:
'''
In console run:
java -jar HMRServer.jar <numer_portu> <ilosc_watkow> e.g.
java -jar HMRServer.jar 9999 24
'''

HOST = "127.0.0.1" #"127.0.0.1"  # The server's hostname or IP address
PORT = 9999 # The port used by the server

def queryHRTDServer(query,max_msg_size=1024):
    query+='\n'
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(bytes(query,'UTF8'))
        fragments = []
        while True: 
            chunk = s.recv(max_msg_size)
            if not chunk: 
                break
            fragments.append(chunk)
        arr = b''.join(fragments)
    return arr.decode('UTF8')

def rem_hmr_files():
    for x in os.listdir():
        if x.endswith(".hmr"):
            os.remove(x)

# Example based on iris dataset without crossvalidation

In [554]:
dataset = 'iris.csv'

In [555]:
data = pd.read_csv('art_datasets/'+dataset)
labels = data['y']
data = data.drop('y', axis = 1)

# labels were removed because one of the clamp's feature is to make clustering, 
# however if necessary you can pass labels in fit function and then clustring stage will be omitted

In [556]:
data.head()

,0,1,2,3
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444


In [557]:
clamp = CLAMP(clusterng_algorithm = KMeans(n_clusters = 3), 
                classification_model = xgb.XGBClassifier(), 
                description_points_ratio=0.01,
                test_size = 0.2,
                metric = 'minkowski',
                thresh = 0.9, 
                bounding_box_selection ='random')
#parameters to adjust there is also possibility to change clustering algorithm and classification model
# available bounding_box_selection parameter: centroids, outliers, tree_query, random

In [558]:
#clamp.fit(data, labels) -- this one is for data with labels
clamp.fit(data) # -- this one is for data without labels

#the dataset has been splited in to train and test dataset, train dataset in provided to anchor to generate rules based on boundingbox method and test dataset is saved to run predict method (check below)

Data without labels, clustering stage implementation
Anchor explainer


In [559]:
predict = clamp.predict(clamp.X_test, clamp.y_test) #labels generation based on the test dataset in generated rules in previous step

Heartdroid run


Loading...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 174.68it/s]


In [560]:
print(clamp.explainer_precision) 
#score calculation (comparison labels from test dataset and those predicted by the clamp -- previous step)
#available: explainer_f1, explainer_accuracy, explainer_classification_report, explainer_recall

0.7296296296296296


In [561]:
r = clamp.justify()
r
rem_hmr_files() # remove temp hmr files
#display genarated rules

,Rule,Precision,Coverage,Cluster
0,1 > 0.50 AND 0 > 0.83,1.0,0.166667,0
1,1 <= 0.50 AND 0 > 0.83,1.0,0.166667,0
2,2 <= -0.94 AND 3 <= -1.02,1.0,0.333333,1
3,2 > -0.94 AND 0 <= 0.83 AND 1 <= -0.25,1.0,0.333333,2
4,2 > 0.42 AND 3 <= 1.22,1.0,0.166667,2


# Grid Search CV

In [545]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [546]:
def scorer(clamp, *args):
    clamp.predict(clamp.X_test, clamp.y_test)
    return {'f1': clamp.explainer_f1, 'accuracy': clamp.explainer_accuracy, 'precision': clamp.explainer_precision}

In [547]:
parameters = [{
                'bounding_box_selection': ['random', 'centroids', 'outliers', 'tree_query'],
                'description_points_ratio': [0.05,0.1,0.2],
                'test_size': [0.2],
                'thresh':[0.9],
                'explainer_type': ['anchor']
},
             
            {
              'explainer_type': ['global'],
              'test_size': [0.2]
}]

list_of_choosen_datasets = ['iris.csv']
list_of_clusters = [3]

In [548]:
%%time
#parameter settings as described above
cv_restuls = []
cv_datasets = []
cv_clf = []

for dataset,cluster_number in zip(list_of_choosen_datasets, list_of_clusters):
    data = pd.read_csv('art_datasets/'+dataset)
    
    labels = data['y']
    data = data.drop('y', axis = 1)
    print(f'Dataset: {dataset}')
    
    clamp = CLAMP(clusterng_algorithm = KMeans(n_clusters = cluster_number), classification_model = xgb.XGBClassifier())
    
    clf = GridSearchCV(clamp, parameters, scoring = scorer, cv = 5, refit = 'precision', n_jobs = 1)
    clf.fit(data)#, labels)
    
    cv_restuls.append(clf.cv_results_)
    cv_datasets.append(dataset)
    cv_clf.append(clf)
    
with open('art_results.pickle', 'wb') as f:
    pickle.dump([cv_restuls, cv_datasets], f)

rem_hmr_files() # remove temp hmr files

Dataset: iris.csv
Data without labels, clustering stage implementation
Anchor explainer
Heartdroid run


Loading...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 160.75it/s]


Data without labels, clustering stage implementation
Anchor explainer



KeyboardInterrupt

